In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

### Above is FIX for Unicode Error in Colab

01. How to run Mistral-7B-Instruct with Langchain and  BitsAndBytes
02. Simple RAG with Langchain / ChromaDB and  Mistral-7B-Instruct
03. Mistral-7B-Instruct with GGUF Format

In [1]:
# !pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip install -q -U einops
# !pip install -q -U safetensors
# !pip install -q -U torch
# !pip install -q -U xformers
# !pip install -q -U langchain
# !pip install -q -U ctransformers[cuda]
# !pip install chromadb
# !pip install sentence-transformers

01 : Running With LangChain

In [ ]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [ ]:
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
model_4bit = AutoModelForCausalLM.from_pretrained( model_id, device_map="auto",quantization_config=quantization_config, )
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
pipeline = pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=1024,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

In [ ]:
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
llm = HuggingFacePipeline(pipeline=pipeline)

In [ ]:
%%time
template = """<s>[INST] You are a helpful, respectful and honest assistant. Answer exactly in few words from the context
Answer the question below from context below :
{context}
{question} [/INST] </s>
"""

question_p = """What is the date for announcement"""
context_p = """ On August 10 said that its arm JSW Neo Energy has agreed to buy a portfolio of 1753 mega watt renewable energy generation capacity from Mytrah Energy India Pvt Ltd for Rs 10,530 crore."""
prompt = PromptTemplate(template=template, input_variables=["question","context"])
llm_chain = LLMChain(prompt=prompt, llm=llm)
response = llm_chain.run({"question":question_p,"context":context_p})
response

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


CPU times: user 1.92 s, sys: 97.6 ms, total: 2.02 s
Wall time: 2.91 s


'The date for the announcement was August 10.'

In [ ]:
%%time
template = """<s>[INST] :
{question} [/INST] </s>
"""

question_p = """What company is buyer and seller here  \n\n\n
                On August 10 said that its arm JSW Neo Energy has agreed to buy a portfolio of 1753 mega watt renewable energy generation capacity from Mytrah Energy India Pvt Ltd for Rs 10,530 crore."""
#context_p = """ On August 10 said that its arm JSW Neo Energy has agreed to buy a portfolio of 1753 mega watt renewable energy generation capacity from Mytrah Energy India Pvt Ltd for Rs 10,530 crore."""

prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=llm)
response = llm_chain.run({"question":question_p})
response

CPU times: user 4.67 s, sys: 0 ns, total: 4.67 s
Wall time: 4.76 s


'\nThe company that is the seller in this transaction is Mytrah Energy India Pvt Ltd., and the company that is the buyer is JSW Neo Energy, a subsidiary of the Japanese steel and trading company JSW Holdings.'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/edu/sample_kidsinmind.csv")

In [ ]:
df_predictions = pd.read_csv("/content/drive/MyDrive/edu/df_predictions(1).csv")

In [ ]:
df_predictions.head(2)

,Language_Score,Predcited_Score,subtitle,language_summary,cleaned_text,inappropriate_words,inappropriate_words_count
0,6,6.543292,The High Council\nhas reached a verdict --guil...,"13 F-words, 1 nearly uttered F-word, 3 obscene...",the high council reached verdict --guilty!you ...,"['bondage', 'bitch', 'ass', 'fuck', 'fuck', 'g...",47
1,7,7.480484,"Danny, give me my shirt!Come and get it.Don't ...","About 25 F-words, 2 obscene hand gestures, 8 s...","danny, shirt!come it.don't think maybe we're t...","['douche', 'damn', 'bitch', 'bra', 'sex', 'lai...",103


In [3]:
from tqdm import tqdm

In [ ]:
df_predictions['mistral_7b'] = None

In [ ]:

for idx in tqdm(range(len(df_predictions))):

    query = "Given the subtitle rating of a movie = "+str(round(df_predictions['Predcited_Score'][idx], 1))+" ; \
     and a list of inappropriate words : "+ df_predictions['inappropriate_words'][idx] + " Generate \
    language summary like kids in mind website? Please consider the below two example language summaries of \
    movies:  \n 1." + df['Language_Description'][0] + "\n 2."+df['Language_Description'][1]

    query = query.replace("[", "")
    query = query.replace("]", "\n\n")


    question_p = query


    prompt = PromptTemplate(template=template, input_variables=["question"])
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    response = llm_chain.run({"question":question_p})

    print("RESPONSE")

    print(response)

    print("*"*100)
    df_predictions['mistral_7b'][idx] = response


  0%|          | 0/201 [00:00<?, ?it/s]<ipython-input-18-cfbb739aa760>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predictions['mistral_7b'][idx] = response
  0%|          | 1/201 [00:19<1:06:28, 19.94s/it]

RESPONSE

Based on the subtitle rating of 6.5, the following language summary can be generated for the movie.

The movie may contain:

* 1-2 F-words or F-word derivatives
* 4-6 scatological terms, including mild ones
* 4-6 anatomical terms
* 1-2 mild obscenities or name-calling terms such as "bitch," "ass," "fuck," "shit," "masturbate," "hell," "prick," "ass," "cock," "toilet," "dead," "skank," "stupid," "bulldyke," "damn," "hell," "fucked," "killing," "stupid," "fat," "ass," "stupid," "tinkle," "ugly," "butt," "stupid," "ass," "piss," "tongue," and "desire."

It's important to note that while this summary is a general guideline, it's not exhaustive and the actual language used in the movie may vary.
****************************************************************************************************


<ipython-input-18-cfbb739aa760>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predictions['mistral_7b'][idx] = response
  1%|          | 2/201 [00:38<1:02:47, 18.93s/it]

RESPONSE
1. This movie contains 1 F-word, 4 scatological terms (1 mild), 4 anatomical terms, 3 mild obscenities, name-calling (freak, psycho, foolish, boring, stubborn, little man), and 4 religious exclamations.
2. This movie includes approximately 67 F-words and their derivatives, 2 obscene hand gestures, 12 sexual references, 38 scatological terms, 38 anatomical terms, 39 mild obscenities, name-calling (crazy, moron, idiots, dumb, sluts, stupid, psycho, punk, Negro, drunks, school boy, boring, pussy whipped, mini-mite, corny, Little Miss Overachiever, homosexual), stereotypes of corporate officials, bosses, single men, single women, Jews, Koreans, Japanese, Irish bar owners, Blacks, lesbians, rich people, and 4 religious obscenities (GD).
****************************************************************************************************


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1154: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
<ipython-input-18-cfbb739aa760>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predictions['mistral_7b'][idx] = response
  1%|▏         | 3/201 [00:58<1:04:43, 19.61s/it]

...

RESPONSE

For a movie with a subtitle rating of 7.1 and a list of inappropriate words, the language summary can be generated as follows:

1. Contains at least one F-word and several scatological terms, mild anatomical terms, and mild obscenities, with name-calling directed towards individuals such as "freak" or "punk".
2. Approximately 67 F-words and its derivatives, several obscene hand gestures, and numerous sexual references, with a significant number of scatological terms, anatomical terms, mild obscenities, and name-calling directed towards individuals such as "crazy", "moron", and "stupid", among others.

It is important to note that language summaries should be tailored to the specific audience or purpose, and that these examples should be used for illustrative purposes only. Additionally, it is important to consider the context of these summaries and the intended audience in order to accurately reflect the language content of the movie.
*****************************************

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1154: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
<ipython-input-18-cfbb739aa760>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predictions['mistral_7b'][idx] = response
100%|█████████▉| 200/201 [46:11<00:16, 16.01s/it]

RESPONSE

Based on the subtitle rating of 6.8 and the list of inappropriate words provided, I would estimate that this movie is rated PG-13 or higher.

The movie likely contains language and themes that are not suitable for younger audiences. The list of inappropriate words includes many vulgarities, sexual references, and derogatory language. It is also worth noting that the list includes words that are associated with violence and death, such as "kill," "dead," and "died."

In addition to the language used in the movie, the themes may also be mature and potentially disturbing. For example, the words "bomb," "hell," and "damn" suggest that the movie deals with dark and potentially violent or traumatic subject matter.

Overall, based on the subtitle rating and the list of inappropriate words provided, I would recommend that this movie is for adults and mature audiences only.
****************************************************************************************************


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1154: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
<ipython-input-18-cfbb739aa760>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predictions['mistral_7b'][idx] = response
100%|██████████| 201/201 [46:18<00:00, 13.82s/it]

RESPONSE

Language Summary:

* This movie contains strong language and inappropriate words such as "devil," "nuke," "ass," "nasty," "hell," and "explosion."
* There are also multiple occurrences of the word "shit" and other offensive words.
* The movie may be unsuitable for younger viewers due to its mature themes and language.
****************************************************************************************************


In [ ]:
df_predictions.to_csv("df_predictions_mistral_7b.csv", index = False)

In [ ]:
from google.colab import files

In [ ]:
files.download('df_predictions_mistral_7b.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
df_predictions = pd.read_csv("df_predictions_mistral_7b.csv")

In [6]:
df_predictions.head(2)

,Language_Score,Predcited_Score,subtitle,language_summary,cleaned_text,inappropriate_words,inappropriate_words_count,mistral_7b
0,6,6.543292,The High Council\nhas reached a verdict --guil...,"13 F-words, 1 nearly uttered F-word, 3 obscene...",the high council reached verdict --guilty!you ...,"['bondage', 'bitch', 'ass', 'fuck', 'fuck', 'g...",47,"\nBased on the subtitle rating of 6.5, the fol..."
1,7,7.480484,"Danny, give me my shirt!Come and get it.Don't ...","About 25 F-words, 2 obscene hand gestures, 8 s...","danny, shirt!come it.don't think maybe we're t...","['douche', 'damn', 'bitch', 'bra', 'sex', 'lai...",103,"1. This movie contains 1 F-word, 4 scatologica..."


In [5]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [7]:

rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

for idx in range(len(df_predictions)):
    
    scores = scorer.score(df_predictions['language_summary'][idx], df_predictions['mistral_7b'][idx])
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rouge2_scores.append(scores['rouge2'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)


In [8]:
sum(rougeL_scores)/len(rougeL_scores)

0.10078826312298081

In [9]:
import bert_score

In [10]:
references = list(df_predictions['language_summary'])
summaries = list(df_predictions['mistral_7b'])

In [11]:
P, R, F1 = bert_score.score(summaries, references, lang='en')
bertscore_precision = P.mean().item()
bertscore_recall = R.mean().item()
bertscore_f1 = F1.mean().item()

print("BERTScore Precision:", bertscore_precision)
print("BERTScore Recall:", bertscore_recall)
print("BERTScore F1:", bertscore_f1)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore Precision: 0.7834903001785278
BERTScore Recall: 0.8227536678314209
BERTScore F1: 0.802058219909668
